<a href="https://colab.research.google.com/github/althemay/RememberMe/blob/main/RememberMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import time
import sqlite3
from pprint import pprint
import glob

# Set the environment variables
os.environ['API_KEY'] = 'tlk_07ESRRT1E0EHV722DYSH80TQ7S9D'
os.environ['API_URL'] = 'https://api.twelvelabs.io/v1.2'

In [ ]:
# Retrieve API URL and API Key from environment variables
API_URL = os.getenv("API_URL")
assert API_URL  # Assert to ensure that API_URL is not None

API_KEY = os.getenv("API_KEY")
assert API_KEY  # Assert to ensure that API_KEY is not None

# Construct the URL for accessing the indexes endpoint of the API
INDEXES_URL = f"{API_URL}/indexes"

# Specify a unique name for the index to be created
INDEX_NAME = "Test3"

# Set up the headers for the HTTP request, including the API key for authorization
headers = {"x-api-key": API_KEY}

# Define the data payload for creating the index, which includes details about the engines and options used
data = {
  "engines": [
    {
      "engine_name": "marengo2.6",  # Specify the first engine name
      "engine_options": ["visual", "conversation", "text_in_video", "logo"]  # Options for the first engine
    },
    {
      "engine_name": "pegasus1",  # Specify the second engine name
      "engine_options": ["visual", "conversation"]  # Options for the second engine
    }
  ],
  "index_name": INDEX_NAME,  # Name of the index
  "addons": ["thumbnail"]  # Additional features (e.g., thumbnail generation)
}

# Send a POST request to the API to create an index with the specified configuration
response = requests.post(INDEXES_URL, headers=headers, json=data)

# Extract the index ID from the response, will be None if not successful
INDEX_ID = response.json().get('_id')

# Print the status code of the response to check if the request was successful
print(f'Status code: {response.status_code}')

# Pretty print the JSON response to see detailed information or error messages
pprint(response.json())


Status code: 201
{'_id': '6634615db38af0b649c4651f'}


In [ ]:
def get_db_connection():
    """Establish and return a connection to the SQLite database."""
    return sqlite3.connect('video_summaries.db')  # Connects to SQLite database file 'video_summaries.db'

def create_summaries_table():
    """Create a table in the SQLite database if it doesn't already exist."""
    with get_db_connection() as conn:  # Use the database connection
        cursor = conn.cursor()  # Create a cursor object using the connection
        # SQL statement to create a table if it doesn't exist
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS summaries (
            video_id TEXT PRIMARY KEY,  # Unique identifier for each video
            summary TEXT,               # Textual summary of the video
            processing_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP  # Automatically stores the current timestamp
        )
        ''')
        conn.commit()  # Commit the changes to the database

create_summaries_table()  # Call the function to ensure the table is created

def insert_or_replace_summary_with_retry(video_id, summary, attempts=3):
    """
    Attempt to insert or replace a summary in the database with a specified number of retry attempts.
    """
    for attempt in range(attempts):  # Loop through the specified number of attempts
        try:
            with get_db_connection() as conn:  # Establish a database connection
                cursor = conn.cursor()  # Create a cursor object
                # SQL statement to insert a new record or update an existing one based on the video_id
                cursor.execute('''
                INSERT INTO summaries (video_id, summary) VALUES (?, ?)
                ON CONFLICT(video_id) DO UPDATE SET summary=excluded.summary;
                ''', (video_id, summary))
                conn.commit()  # Commit the changes to the database
                print(f"Summary for Video ID: {video_id} saved to the database.")
                return True  # Return True to indicate success
        except sqlite3.OperationalError as e:  # Catch and handle specific SQLite operational errors
            print(f"Attempt {attempt+1}: Database operation failed with error: {e}")  # Print the error and attempt number
            time.sleep(0.5)  # Wait for half a second before retrying

    return False  # Return False if all attempts fail

In [ ]:
# Function to submit a video URL for processing by an external provider
def submit_video_for_processing(video_url):
    TASKS_URL = f"{API_URL}/tasks/external-provider"  # Define the URL for submitting tasks
    data = {"index_id": INDEX_ID, "url": video_url}  # Set up the data payload with index ID and video URL
    response = requests.post(TASKS_URL, headers=headers, json=data)  # Make the POST request
    if response.status_code == 201:
        task_id = response.json().get("_id")  # Retrieve the task ID from the response
        print(f"Task submitted successfully. Task ID: {task_id}")
        return task_id  # Return the task ID
    else:
        print(f"Failed to submit task: {response.status_code}")  # Print failure message
        pprint(response.json())
        return None  # Return None if submission fails

# Function to wait for the completion of the task
def wait_for_task_completion(task_id):
    TASK_STATUS_URL = f"{API_URL}/tasks/{task_id}"  # Define the URL to check task status
    while True:
        response = requests.get(TASK_STATUS_URL, headers=headers)  # Poll the status of the task
        if response.status_code == 200:
            status = response.json().get("status")  # Get the status from the response
            if status == "ready":
                video_id = response.json().get('video_id')  # Get the video ID once the task is ready
                print(f"Task completed. Video ID: {video_id}")
                return video_id  # Return the video ID
            elif status == "failed":
                print("Task failed.")  # Print failure message
                return None  # Return None if the task failed
        else:
            print(f"Failed to get task status: {response.status_code}")  # Print error if status check fails
            return None
        time.sleep(10)  # Sleep for 10 seconds before polling again


def get_video_summary(video_id):
    GENERATE_URL = f"{API_URL}/generate"  # Define the URL to generate the summary
    data = {"video_id": video_id, "prompt": "Make a summary"}  # Set up the data payload
    response = requests.post(GENERATE_URL, headers=headers, json=data)  # Make the POST request
    if response.status_code == 200:
        summary = response.json().get('data')  # Get the summary data from the response
        print("Video summary generated successfully.")
        return summary  # Return the summary
    else:
        print(f"Failed to generate summary: {response.status_code}")  # Print failure message
        pprint(response.json())
        return None  # Return None if summary generation fails

# Main function to run the program
def main():
    video_url = input("Enter the video URL: ")  # Prompt user to enter the video URL
    task_id = submit_video_for_processing(video_url)  # Submit the video for processing
    if task_id:
        video_id = wait_for_task_completion(task_id)  # Wait for the task to complete
        if video_id:
            summary = get_video_summary(video_id)  # Get the summary for the video
            if summary:
                if not insert_or_replace_summary_with_retry(video_id, summary):
                    print("Failed to save summary to the database.")  # Try to save the summary to the database and print a message if it fails

# Function to retrieve and print all summaries from the database
def get_summaries():
    try:
        with get_db_connection() as conn:  # Establish database connection
            cursor = conn.cursor()  # Create a cursor object
            cursor.execute('SELECT * FROM summaries')  # Execute a query to retrieve all summaries
            rows = cursor.fetchall()  # Fetch all rows of the query result
            if rows:
                for row in rows:
                    print(row)  # Print each row
            else:
                print("No summaries found in the database.")  # Print if no summaries found
    except sqlite3.Error as e:
        print(f"An error occurred when retrieving summaries: {e}")  # Print any errors encountered

if __name__ == "__main__":
    main()  # Execute the main function if the script is run as a standalone program

Enter the video URL: https://www.youtube.com/watch?v=YKsQJVzr3a8&ab_channel=FreeSubsOutTheAss
Task submitted successfully. Task ID: 66346339b38af0b649c46529
Task completed. Video ID: 66346346d1cd5a287c957b64
Video summary generated successfully.
Summary for Video ID: 66346346d1cd5a287c957b64 saved to the database.


In [ ]:
get_summaries()


('6634619ed1cd5a287c957b63', 'The video presents a straightforward countdown sequence, elegantly designed with a classic aesthetic. It begins with the number 5 and methodically counts down to 1. Each number is prominently displayed at the center of the screen, encircled by a circular border that features smaller, evenly spaced numbers ranging from 1 to 60, reminiscent of a clock face. The background is kept to a simple, uniform gray, ensuring that the focus remains on the countdown itself. The use of black for the numbers against the gray backdrop offers a stark, visually appealing contrast. This type of sequence is typically utilized in media productions to signify a transition or the impending start of an event, providing viewers with a clear, visual cue that something is about to commence.', '2024-05-03 04:05:21')
('66346346d1cd5a287c957b64', "The video presents a poignant scene set in an urban residential area, capturing a moment where an individual, dressed in a green shirt and kh